In [ ]:
pip install TA-Lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.5/368.5 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib)


In [ ]:
import yfinance as yf

# Fetch historical data for Bitcoin (or any stock)
ticker = 'BTC-USD'
bitcoin_data = yf.download(ticker, start='2017-08-17', end='2024-09-15')

# 1. Calculate RSI to detect overconfidence or panic selling
#bitcoin_data['RSI'] = talib.RSI(bitcoin_data['Close'], timeperiod=14)

# 2. Calculate Moving Averages (SMA/EMA) to detect herd behavior
bitcoin_data['SMA_20'] = bitcoin_data['Close'].rolling(window=20).mean()  # 20-day Simple Moving Average
bitcoin_data['EMA_50'] = bitcoin_data['Close'].ewm(span=50, adjust=False).mean()  # 50-day Exponential Moving Average

# 3. Calculate volatility to detect fear and uncertainty
bitcoin_data['volatility'] = bitcoin_data['Close'].pct_change().rolling(window=20).std()

# 4. Bollinger Bands to detect fear and greed
bitcoin_data['middle_band'] = bitcoin_data['Close'].rolling(window=20).mean()
bitcoin_data['upper_band'] = bitcoin_data['middle_band'] + 2*bitcoin_data['Close'].rolling(window=20).std()
bitcoin_data['lower_band'] = bitcoin_data['middle_band'] - 2*bitcoin_data['Close'].rolling(window=20).std()

# Drop rows where the calculated indicators are NaN
bitcoin_data.dropna(inplace=True)

# Display the first few rows after dropping NaN values
print(bitcoin_data[['Close', 'SMA_20', 'volatility', 'upper_band', 'lower_band']].head())


[*********************100%***********************]  1 of 1 completed

                  Close       SMA_20  volatility   upper_band   lower_band
Date                                                                      
2017-09-06  4597.120117  4381.569507    0.035017  4848.073956  3915.065057
2017-09-07  4599.879883  4403.532495    0.033539  4867.592096  3939.472894
2017-09-08  4228.750000  4405.284985    0.038671  4866.262582  3944.307388
2017-09-09  4226.060059  4412.204993    0.038174  4856.976914  3967.433071
2017-09-10  4122.939941  4418.264990    0.038291  4842.308949  3994.221031


In [ ]:
import requests
import pandas as pd

# Function to fetch GDELT data
def fetch_gdelt_data(start_date, end_date, keyword):
    url = f"https://api.gdeltproject.org/api/v2/doc/doc?query={keyword}&mode=artlist&maxrecords=100&format=json&startdatetime={start_date}&enddatetime={end_date}"
    response = requests.get(url)

    # Print the raw response to see what the API is returning
    print(f"Status Code: {response.status_code}")
    print(f"Response Text: {response.text}")  # Print the raw response

    if response.status_code == 200:
        try:
            data = response.json()['articles']
            return pd.DataFrame(data)
        except ValueError as e:
            print(f"Error decoding JSON: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if there's a JSON decoding issue
    else:
        print(f"Error: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if the request fails

# Example parameters
start_date = "2017-09-07"
end_date = "2024-09-15"
keyword = "Bitcoin"

# Fetch the GDELT data
gdelt_df = fetch_gdelt_data(start_date, end_date, keyword)


Status Code: 200
Response Text: Invalid query start date format (must be in YYYYMMDDHHMMSS format).

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
